## My Trainer

In [1]:
%cd /home/q123/Desktop/explo/

from mushroom_rl.environments.dm_control_env import DMControl
from mushroom_rl.policy import DeterministicPolicy
from src.ddpg import DDPG
from src.helpers import setup_experiment
from src.config import get_configs
import torch
from src.approximators.actor import ActorNetwork
from src.approximators.critic import CriticNetwork

import torch.nn.functional as F

# MDP
horizon = 500
gamma = 0.99
gamma_eval = 1.
#mdp = DMControl('walker', 'stand', horizon, gamma)


# Settings
initial_replay_size = 500
max_replay_size = 5000
batch_size = 200
n_features = 80
tau = .001

/home/q123/Desktop/explo


/home/q123/miniconda3/envs/boptim/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/q123/miniconda3/envs/boptim/lib/python3.10/site-packages/gym/envs/registration.py:415: UserWarning: WARN: The `registry.env_specs` property along with `EnvSpecTree` is deprecated. Please use `registry` directly as a dictionary instead.
  logger.warn(
<frozen importlib._bootstrap>:283: DeprecationWarning: the load_module() method is deprecated and slated for removal in Python 3.12; use exec_module() instead
pybullet build time: Jun 23 2022 12:25:14


In [2]:
import multiprocessing as mp
from copy import deepcopy
import torch


class ESQOptimizer(object):
    
    
    def __init__(self,critic,actor,
                        sigma,params_per_step,n_workers=None):
        
        if n_workers is None : n_workers = mp.cpu_count()
        
        self.args = locals()
        self.args.pop("self")
        
        
        self.actor = actor
        self.optimizer = torch.optim.Adam(actor.model.network.parameters())
        #self.__dict__.update(locals())
        

    def run_noisy_advantage(self,states,critic,actor,
                            sigma,seed):
        
        torch.manual_seed(seed)
        n_params = actor.model.network.n_params
        eps = torch.randn(n_params) * sigma
        actor.model.network.add_noise(eps)
        #actor2 = actor.model.network.add_noise(-eps)
    
        noisy_actions = actor(states,output_tensor=True)
        noisy_q = critic(states,noisy_actions,output_tensor=True) ##  add absorbing flag
        noisy_q = torch.sum(noisy_q)
        noisy_grad = torch.autograd.grad(noisy_q,actor.model.network.parameters()) ## hotfix
        
        #return grad1+grad2
        
        return noisy_grad
        
    def run_parallel_advantage(self,states,critic,actor,
                               sigma,params_per_step,n_workers):
        
        args = [(states,critic,actor,sigma,seed) for seed in range(params_per_step)]
        
        ctx = mp.get_context('spawn')
        
        # Step 1: Init multiprocessing.Pool()

        with ctx.Pool(n_workers) as pool:
        
            # Step 2:  Run processes (we might need to use mapreduce to avoid big memory usage)
            grads = pool.starmap(self.run_noisy_advantage,args) ## list of [(reward*eps)]

            # Step 3: Wait for workers to run then close pool
            pool.close()
            pool.join()
            
        
        return grads
    
    def compute_grads(self,states):
        
        grads = self.run_parallel_advantage(**self.args,states=states)  
        
        # Step 4: Aggregate gradients
        
        grad_stack = []
        
        for i in range(len(grads[0])):
            
            grad_stack.append(torch.stack([grad[i] for grad in grads]))
            
        grads = tuple(torch.sum(stack,dim=0) for stack in grad_stack)
        
        return grads

    def step(self,states):
        
        self.optimizer.zero_grad()  
        actor_grad = self.compute_grads(states)      
        self.actor.model.network.grad = actor_grad ## optimizer usually minimizes (add -)
        self.optimizer.step()
      

In [3]:
def critic_step(agent,transitions):
    
    agent._replay_memory.add(transitions)

    state, action, reward, next_state, absorbing, _ =\
        agent._replay_memory.get(agent._batch_size())

    q_next = agent._next_q(next_state, absorbing)
    q_target = reward + agent.mdp_info.gamma * q_next

    agent._critic_approximator.fit(state, action, q_target,
                                    **agent._critic_fit_params)
    
    agent._update_target(agent._critic_approximator,
                        agent._target_critic_approximator)

In [4]:
%cd /home/q123/Desktop/explo/src

env_name = "Swimmer-v4"
kernel_name = "rbf"


## Setup environment
env_config,likelihood_config,kernel_config,optimizer_config,trainer_config = get_configs(env_name,kernel_name)
_,env = setup_experiment(env_config,kernel_config,likelihood_config,additional_layers=[])


# Setup DDPG
mdp = env.env

policy_class = DeterministicPolicy
policy_params = dict()

actor_input_shape = mdp.info.observation_space.shape
actor_params = dict(network=ActorNetwork,
                    n_features=n_features,
                    input_shape=actor_input_shape,
                    output_shape=mdp.info.action_space.shape)

actor_optimizer = {'class': torch.optim.Adam,
                   'params': {'lr': 1e-5}}

critic_input_shape = (actor_input_shape[0] + mdp.info.action_space.shape[0],)
critic_params = dict(network=CriticNetwork,
                     optimizer={'class': torch.optim.Adam,
                                'params': {'lr': 1e-3}},
                     loss=F.mse_loss,
                     n_features=n_features,
                     input_shape=critic_input_shape,
                     output_shape=(1,))

agent = DDPG(mdp.info, policy_class,policy_params,
             actor_params, actor_optimizer, 
             critic_params,
             batch_size, initial_replay_size, max_replay_size,
             tau)

esq_optimizer = ESQOptimizer(
                         critic=agent._critic_approximator,
                         actor = agent._actor_approximator,
                         sigma=1,
                         params_per_step=1,
                         n_workers=1)

for i in range(1):
    
    
    _,states,transitions = env.run_many(agent._actor_approximator,1)
    print("done collecting states")
    critic_step(agent,transitions)
    print("done training critic")
    grads = esq_optimizer.step(states[:10])
    
    #print(optimizer.actor.parameters())

/home/q123/Desktop/explo/src
MathLog.src.helpers : WARNING : MLP dimensions : [8, 2]


/home/q123/Desktop/explo/src/environments/objective.py:207: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  rslt = torch.tensor(state, dtype=torch.float32)
/home/q123/Desktop/explo/src/environments/objective.py:100: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  states.append(torch.tensor(self.manipulate_state(state)))


Using ard_num_dims = 18
done collecting states
done training critic


/home/q123/miniconda3/envs/boptim/lib/python3.10/site-packages/mushroom_rl/approximators/parametric/torch_approximator.py:166: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  np.int)
Process SpawnPoolWorker-1:
Traceback (most recent call last):
  File "/home/q123/miniconda3/envs/boptim/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/q123/miniconda3/envs/boptim/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/q123/miniconda3/envs/

KeyboardInterrupt: 

In [ ]:
# from mushroom_rl.approximators import Regressor
# from mushroom_rl.approximators.parametric import TorchApproximator
# from mushroom_rl.utils.replay_memory import ReplayMemory

# replay_memory = ReplayMemory(initial_replay_size, max_replay_size)
# critic_approximator = Regressor(TorchApproximator,
#                                               **critic_params)


In [ ]:
# optimizer = ESOptimizer(env,torch.zeros(env.mlp.len_params),sigma=1e-2,
#                 params_per_step=50,episodes_per_param=1,n_workers=8)
# # for i in range(1):
    
# #     optimizer.step()
    
# #     if i % 3 == 0:
# #         avg_reward,_,transitions = env.run_many(optimizer.policy_params,5)
# #         # print(f'avg_rewarad {avg_reward} ')
# #         # print(f'policy_params : {optimizer.policy_params}')
